In [85]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [86]:
# Paso 1: Cargar los datos
df = pd.read_csv('movies_dataset.csv', low_memory=False)
data = df.copy()

In [87]:
data = data.drop_duplicates(subset = 'title')

In [88]:
#Calculamos la media de la columna de promedio de votos
C = data['vote_average'].mean()
print(C)

5.624940274853939


In [89]:
# Calculamos el número mínimo de votos necesarios para estar en la tabla, m
m = data['vote_count'].quantile(0.90)
print(m)

158.0


In [90]:
# Filtramos todas las películas calificadas en un nuevo DataFrame
data = data.loc[data['vote_count'] >= m]

In [91]:
# Función que calcula la calificación ponderada de cada película
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

Sacamos el promedio ponderado con esta formula

            (WR) = (v/(v + m))R + (m/(v+m))C

R = promedio de la pelicula (Media)  <br>
v = número de votos para la película <br>
m = votos mínimos necesarios para figurar en el Top 50 (actualmente 1000) <br>
C = la media de votos en todo el informe (actualmente 6,8)

In [92]:
# Definimos una nueva característica 'score' y calculamos su valor con `weighted_rating()
data['score'] = data.apply(weighted_rating, axis=1)

In [93]:
# Ordenamos las películas según la columna de puntuación
data = data.sort_values('score', ascending = False)

In [94]:
# Exploramos el overview de las películas
data['overview']

314      Framed in the 1940s for the double murder of h...
10309    Raj is a rich, carefree, happy-go-lucky second...
834      Spanning the years 1945 to 1955, a chronicle o...
12481    Batman raises the stakes in his war on crime. ...
2843     A ticking-time-bomb insomniac and a slippery s...
                               ...                        
9710     Tim Avery, an aspiring cartoonist, finds himse...
12911    In DISASTER MOVIE, the filmmaking team behind ...
3471     In the year 3000, man is no match for the Psyc...
11557    When Edward, Peter, Lucy and Susan each follow...
13566    The young warrior Son Goku sets out on a quest...
Name: overview, Length: 4241, dtype: object

In [95]:
# Creamos el objeto TF-IDF Vectorizer y eliminamos las palabras vacías en inglés
tfidf = TfidfVectorizer(stop_words='english')

In [96]:
#Reemplazmos NaN con una cadena vacia
data['overview'] = data['overview'].fillna('')

In [97]:
# Creamos la matriz
tfidf_matrix = tfidf.fit_transform(data['overview'])

In [98]:
# Asignamos índices enteros de características al nombre de la característica#
tfidf.get_feature_names_out()[5000:5010]

array(['disguise', 'disguised', 'disguises', 'disgusted', 'dishes',
       'dishonored', 'disillusioned', 'disillusionment', 'disillusions',
       'disintegrate'], dtype=object)

In [99]:
# Calculamos la puntuación de similitud de coseno
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [100]:
#Restablecemos los índices para que coincidan con la matriz de similitud de coseno
data.reset_index(drop = True, inplace = True)

In [101]:
# Construimos un mapa inverso de índices y títulos de películas.
index = pd.Series(data.index, index = data['title']).drop_duplicates()

In [103]:
# Creamos la función de recomendación 
def recomendacion(titulo, cosine_sim = cosine_sim):

  if titulo not in index:
    return "La película no se encuentra entre el 10% de las mejores películas. Intenta con una mejor!"

  idx = index[titulo]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
  sim_scores = sim_scores[1:6]
  movie_indices = [i[0] for i in sim_scores]
  result = data['title'].iloc[movie_indices]
  return {"Te recomendamos estas peliculas" : list(result)}

In [104]:
recomendacion("Superman")

{'Te recomendamos estas peliculas': ['Superman II',
  'Superman Returns',
  'Superman IV: The Quest for Peace',
  "National Lampoon's Christmas Vacation",
  'Central Intelligence']}